# Kalshi BTC Paper Trading Analysis

This notebook analyzes the performance of different trading strategies on Kalshi hourly BTC markets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')

from src.config import SimulationConfig
from src.simulator import Simulator
from src.strategies.no_trade import NoTradeStrategy
from src.strategies.momentum import MomentumStrategy
from src.strategies.mean_reversion import MeanReversionStrategy
from src.metrics import MetricsCalculator

## 1. Load Data

In [ ]:
# Load BTC prices
btc_df = pd.read_csv('../data/btc_prices_minute.csv')
btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'])

print(f"BTC Price Data: {len(btc_df)} rows")
print(f"Time range: {btc_df['timestamp'].min()} to {btc_df['timestamp'].max()}")
print(f"Price range: ${btc_df['price'].min():.2f} to ${btc_df['price'].max():.2f}")
btc_df.head()

In [ ]:
# Load markets
markets_df = pd.read_csv('../data/kalshi_markets.csv')
markets_df['hour_start'] = pd.to_datetime(markets_df['hour_start'])

print(f"Markets: {len(markets_df)} total")
print(f"Hours: {markets_df['hour_start'].nunique()}")
print(f"Strikes per hour: {len(markets_df) // markets_df['hour_start'].nunique()}")
markets_df.head(10)

## 2. Visualize BTC Price Movement

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(btc_df['timestamp'], btc_df['price'], linewidth=1.5)
plt.title('BTC Price (Minute-by-Minute)', fontsize=14, fontweight='bold')
plt.xlabel('Time')
plt.ylabel('Price ($)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Run Simulations

In [ ]:
# Configuration
config = SimulationConfig(
    starting_balance=10000.0,
    max_position_pct=0.1,
    fee_per_contract=0.0,
    btc_price_interval=250,
    btc_prices_path="../data/btc_prices_minute.csv",
    markets_path="../data/kalshi_markets.csv",
    contract_prices_path="../data/kalshi_contract_prices.csv"
)

# Initialize simulator
simulator = Simulator(config)

# Define strategies
strategies = [
    NoTradeStrategy(),
    MomentumStrategy(lookback_minutes=3, max_position_pct=0.1),
    MeanReversionStrategy(window_minutes=10, threshold=0.05, max_position_pct=0.1)
]

# Run simulations
all_results = []
for strategy in strategies:
    print(f"Running {strategy.name}...")
    results = simulator.run(strategy)
    all_results.append(results)
    
print("\nSimulations complete!")

## 4. Compare Strategy Performance

In [ ]:
# Create comparison table
comparison = MetricsCalculator.create_comparison_table(all_results)
comparison

## 5. Visualize Performance

In [ ]:
# Plot returns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total PnL
axes[0, 0].bar(comparison['strategy_name'], comparison['total_pnl'])
axes[0, 0].set_title('Total PnL by Strategy', fontweight='bold')
axes[0, 0].set_ylabel('PnL ($)')
axes[0, 0].grid(True, alpha=0.3)

# Win Rate
axes[0, 1].bar(comparison['strategy_name'], comparison['win_rate'])
axes[0, 1].set_title('Win Rate by Strategy', fontweight='bold')
axes[0, 1].set_ylabel('Win Rate (%)')
axes[0, 1].grid(True, alpha=0.3)

# Total Trades
axes[1, 0].bar(comparison['strategy_name'], comparison['total_trades'])
axes[1, 0].set_title('Total Trades by Strategy', fontweight='bold')
axes[1, 0].set_ylabel('Number of Trades')
axes[1, 0].grid(True, alpha=0.3)

# Max Drawdown
axes[1, 1].bar(comparison['strategy_name'], comparison['max_drawdown'])
axes[1, 1].set_title('Max Drawdown by Strategy', fontweight='bold')
axes[1, 1].set_ylabel('Drawdown (%)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Detailed Trade Analysis

In [ ]:
# Analyze trades for the best performing strategy
best_strategy_idx = comparison['total_pnl'].idxmax()
best_result = all_results[best_strategy_idx]
best_portfolio = best_result['portfolio']

print(f"Best Strategy: {best_result['strategy_name']}")
print(f"Total PnL: ${best_result['total_pnl']:.2f}")
print(f"\nTrade History:")

if best_portfolio.pnl_history:
    pnl_df = pd.DataFrame(best_portfolio.pnl_history)
    print(f"Total Trades: {len(pnl_df)}")
    print(f"Wins: {(pnl_df['pnl'] > 0).sum()}")
    print(f"Losses: {(pnl_df['pnl'] < 0).sum()}")
    pnl_df.head(10)

## 7. Equity Curve

In [ ]:
# Plot equity curves for all strategies
plt.figure(figsize=(14, 6))

for result in all_results:
    hours = result['hours_traded']
    if hours:
        equity = [result['initial_balance']]
        for hour in hours:
            equity.append(hour['portfolio_value'])
        
        plt.plot(range(len(equity)), equity, marker='o', label=result['strategy_name'], linewidth=2)

plt.title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Hour')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()